In [ ]:
# Get the uniprot mapping IDs
# ! wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/HUMAN_9606_idmapping.dat.gz

# Grep mapping ids to string only
# ! grep STRING HUMAN_9606_idmapping.dat > string.map


# Get string interactions
# ! wget https://stringdb-static.org/download/protein.links.v11.5/9606.protein.links.v11.5.txt.gz

In [ ]:
import pandas as pd

In [ ]:
dfmap = pd.read_csv('../../Downloads/string.map', sep='\t', names=['protid', 'db', 'ensmbl'])
dfstring = pd.read_csv('../../Downloads/9606.protein.links.v11.5.txt', sep=' ')
df = pd.read_csv('tmp/final_results_all.tsv', sep='\t')

In [ ]:
dfmap.head()

In [ ]:
dfstring.head()

In [ ]:
df.head()

In [ ]:
#Getting second order interactions
df = df[(df.order ==2) & df.coef_id.apply(lambda x: ':' in x)]
df['prots'] = df.snps.apply(lambda x: x.split('.')[0]+'+'+x.split('+')[1].split('.')[0])
df.head()

In [ ]:
# Applying bonferroni for multiple testing
from  statsmodels.stats.multitest import multipletests
df['p_corrected'] = multipletests(df['P>|z|'], alpha=0.05, method='bonferroni')[1]


In [ ]:
# Mapping DB string to uniprot IDs
dfmapped = dfstring.merge(dfmap, left_on='protein1', right_on='ensmbl').merge(dfmap, left_on='protein2', right_on='ensmbl')

#Making the interactions the same way my tool does them
dfmapped = dfmapped[['protein1','protein2','combined_score','protid_x','protid_y']]
dfmapped['combined_col'] = dfmapped.apply(lambda row:'+'.join(sorted([row['protid_x'], row['protid_y']])), axis=1) #inefficient, takes a while. Probably faster with if, else function
dfmapped.head()

In [ ]:
#Combining string and RF
dfcombi=df.merge(dfmapped, left_on='prots', right_on='combined_col')

In [ ]:
print('RF all interactions',len(df))
print('RF all interactions NOT REPEATED',len(df.drop_duplicates('snps',keep='first')))
print('Bonferroni corrected interactions', sum(df['p_corrected']<0.05))
print('overlapping string&RF interactions',len(dfcombi))
print('overlapping string&RF interactions NOT REPEATED',len(dfcombi.drop_duplicates('snps',keep='first')))
print('overlapping string&RF SIGNIFICANT interactions', sum(dfcombi['p_corrected']<0.05))
print('overlapping string&RF SIGNIFICANT interactions NOT REPEATED', sum((dfcombi.drop_duplicates('snps',keep='first')['p_corrected']<0.05)))